In [1]:
import plotly.express as px
import pandas as pd
from datetime import datetime as dt, timedelta
import json

In [2]:
stanowiska = [{}]

In [3]:
with open("technologia.json", "r") as f:
    tehnologia = json.loads(f.read())["tech"]

tech = pd.DataFrame(tehnologia)
tech

,Nazwa,idop,operacja_nazwa,norma,asortyment_powiazany,id_wejsciowe,opuznienie startu nastepnej operacji,przelicznik z poprzedniej operacji,id_wyjsciowe,rodzaj_startu,start_przy_zapasie_z_porzedniej_operacji
0,Prod_1,100,przegotowanie form,65min/10000szt,Asor_1,0,0,None,200,None,None
1,Prod_1,200,formowanie reczne,100szt/h,Asor_1,100,130m,1,400,None,None
2,Prod_1,300,giecie preta,400szt/h,Asor_1,0,0,None,400,None,None
3,Prod_1,400,klenie uszek reczne,120szt/h,Asor_1,200,0,1,600,None,None
4,Prod_1,400,klenie uszek reczne,120szt/h,Asor_1,300,0,1,600,None,None
5,Prod_1,500,ciecie kartonu,130kpl/h,Szklanka Caffelatte 200ml kpl 6szt,0,0,None,600,None,None
6,Prod_1,600,pakowanie,250kpl/h,Szklanka Caffelatte 200ml kpl 6szt,400,0,6szt/kpl,700,None,None
7,Prod_1,600,pakowanie,250kpl/h,Szklanka Caffelatte 200ml kpl 6szt,500,0,1,700,None,None
8,Prod_1,700,zaladunek,600kpl/h,Szklanka Caffelatte 200ml kpl 6szt,600,0,1,700,None,None


In [4]:
zamowienia = [
   dict(Numer_zamowienia= "2022_1", Produkt= "Prod_1", Ilosc=1500, Data_dostawy=dt(2023,4,15,13)),
   # dict(Numer_zamowinie= "Zam0", Produkt= "Prod_2", Ilosc=3000, Data_dostawy=dt(2023,5,15)),
   dict(Numer_zamowinie= "2022_2", Produkt= "Prod_1", Ilosc=3000, Data_dostawy=dt(2023,4,25,13)),
   # dict(Numer_zamowinie= "Zam2", Produkt= "Prod_2", Ilosc=5000, Data_dostawy=dt(2023,5,15)),
   dict(Numer_zamowinie= "2022_3", Produkt= "Prod_1", Ilosc=3000, Data_dostawy=dt(2023,9,15,13)),
]
zamowienia[0]

{'Numer_zamowienia': '2022_1',
 'Produkt': 'Prod_1',
 'Ilosc': 1500,
 'Data_dostawy': datetime.datetime(2023, 4, 15, 13, 0)}

In [5]:
normy = pd.DataFrame(tech[tech["Nazwa"] == zamowienia[0]["Produkt"]][["idop","operacja_nazwa", "norma", "przelicznik z poprzedniej operacji", "id_wejsciowe", "id_wyjsciowe"]])

normy["ilosc"] = normy["norma"].apply(lambda x: int(x.split("/")[0][:-3]))
normy["jednostka"] = normy["norma"].apply(lambda x: x.split("/")[0][-3:])
normy.set_index("idop", inplace=True)
normy

,operacja_nazwa,norma,przelicznik z poprzedniej operacji,id_wejsciowe,id_wyjsciowe,ilosc,jednostka
idop,,,,,,,
100,przegotowanie form,65min/10000szt,None,0,200,65,min
200,formowanie reczne,100szt/h,1,100,400,100,szt
300,giecie preta,400szt/h,None,0,400,400,szt
400,klenie uszek reczne,120szt/h,1,200,600,120,szt
400,klenie uszek reczne,120szt/h,1,300,600,120,szt
500,ciecie kartonu,130kpl/h,None,0,600,130,kpl
600,pakowanie,250kpl/h,6szt/kpl,400,700,250,kpl
600,pakowanie,250kpl/h,1,500,700,250,kpl
700,zaladunek,600kpl/h,1,600,700,600,kpl


In [18]:
def ustaw_operacje(zam, technologia, kierunek): 
    """
    zam -> pozycja zamowienia

    technologia -> kopia DataFrame z odpowiednia technologia
    
    kierunek -> kierunek przydzilu operacji 
                1: do przodu
                -1: do tylu
    
    """
    op = list()
    
    for _idop in technologia.idop.unique()[::kierunek]:

        if kierunek == -1:
            
            op.append(
    
             dict(Zamowienie=zam["Numer_zamowienia"], 
             Produkt=list(technologia.Nazwa.unique())[0], 
             Operacja= technologia[technologia.idop == _idop].operacja_nazwa.unique()[0],
             stanowisko=None, 
             _start=None, _end=None)
            
            )
    
    
    
    
    
    return op

pd.DataFrame(ustaw_operacje(zamowienia[0],tech[tech.Nazwa == "Prod_1"].copy(),-1))
# o[0]["_start"] = dt(2023,3,30)
# o[0]["ilosc"] = 231
# o


[{'Zamowienie': '2022_1',
  'Produkt': 'Prod_1',
  'Operacja': 'zaladunek',
  'stanowisko': None,
  '_start': datetime.datetime(2023, 3, 30, 0, 0),
  '_end': None,
  'ilosc': 231},
 {'Zamowienie': '2022_1',
  'Produkt': 'Prod_1',
  'Operacja': 'pakowanie',
  'stanowisko': None,
  '_start': None,
  '_end': None},
 {'Zamowienie': '2022_1',
  'Produkt': 'Prod_1',
  'Operacja': 'ciecie kartonu',
  'stanowisko': None,
  '_start': None,
  '_end': None},
 {'Zamowienie': '2022_1',
  'Produkt': 'Prod_1',
  'Operacja': 'klenie uszek reczne',
  'stanowisko': None,
  '_start': None,
  '_end': None},
 {'Zamowienie': '2022_1',
  'Produkt': 'Prod_1',
  'Operacja': 'giecie preta',
  'stanowisko': None,
  '_start': None,
  '_end': None},
 {'Zamowienie': '2022_1',
  'Produkt': 'Prod_1',
  'Operacja': 'formowanie reczne',
  'stanowisko': None,
  '_start': None,
  '_end': None},
 {'Zamowienie': '2022_1',
  'Produkt': 'Prod_1',
  'Operacja': 'przegotowanie form',
  'stanowisko': None,
  '_start': None,
  '

In [7]:
zam1 = "2023_1"
koniec_ostaniej_operacji = zamowienia[0]["Data_dostawy"]-timedelta(3)
start_zal = koniec_ostaniej_operacji - timedelta(hours=zamowienia[0]["Ilosc"]/normy.iloc[-1]["ilosc"])
# koniec_pak =  
zal1 = [start_zal, koniec_ostaniej_operacji]
pak1 = [zal1[0]-timedelta(hours=(zamowienia[0]["Ilosc"]/250)),zal1[0]]
cie_k1 = [pak1[0] - timedelta(hours=(zamowienia[0]["Ilosc"]/130)) ,pak1[0]]
klu_r1 = [pak1[0]-timedelta(hours=25),pak1[0]]
klu_r2 = [pak1[0]-timedelta(hours=25),pak1[0]]
klu_r3 = [pak1[0]-timedelta(hours=25),pak1[0]]
gip_r1 = [klu_r1[0]-timedelta(hours=(1500*6/400)), klu_r1[0]]
frw1_s1 = [gip_r1[0]-timedelta(hours=750*6/100),gip_r1[0]]
frw1_s2 = [gip_r1[0]-timedelta(hours=750*6/100),gip_r1[0]]

to1 = [
   dict(Zamowienie=zam1, Produkt="Prod_1", Operacja="zaladunek", stanowisko="zaladunek_1", 
        Start=zal1[0], End=zal1[1],Ilosc=""),
     
   dict(Zamowienie=zam1, Produkt="Prod_1", Operacja="pakowanie", stanowisko="pakowanie h1s1", 
        Start=pak1[0], End=pak1[1],Ilosc="1500kpl"),
    
   dict(Zamowienie=zam1, Produkt="Prod_1", Operacja="ciecie kartonu", stanowisko="ciecie kartonu", 
        Start=cie_k1[0], End=cie_k1[1],Ilosc="1500kpl"),
        
   dict(Zamowienie=zam1, Produkt="Szklanka late 200ml", Operacja="klejenie uszek reczne", stanowisko="klejnie uszek r3", 
        Start=klu_r1[0], End=klu_r1[1],Ilosc="3000szt"),

   dict(Zamowienie=zam1, Produkt="Szklanka late 200ml", Operacja="klejenie uszek reczne", stanowisko="klejnie uszek r2", 
        Start=klu_r1[0], End=klu_r1[1],Ilosc="3000szt"),

   dict(Zamowienie=zam1, Produkt="Szklanka late 200ml", Operacja="klejenie uszek reczne", stanowisko="klejnie uszek r1", 
        Start=klu_r1[0], End=klu_r1[1],Ilosc="3000szt"),
    
   dict(Zamowienie=zam1, Produkt="Pret 5mm bezbarwny", Operacja="giecie preta", stanowisko="giecie preta", 
        Start=gip_r1[0], End=gip_r1[1],Ilosc="9000szt"),
   
   dict(Zamowienie=zam1, Produkt="Szklanka late 200ml", Operacja="formowanie reczne", stanowisko="w1s2", 
        Start=frw1_s1[0], End=frw1_s1[1],Ilosc="3000szt"),    

   dict(Zamowienie=zam1, Produkt="Szklanka late 200ml", Operacja="formowanie reczne", stanowisko="w1s1", 
        Start=frw1_s2[0], End=frw1_s2[1],Ilosc="3000szt"),    

   dict(Zamowienie=zam1, Produkt="Szklanka late 200ml", Operacja="przygotowanie formy", stanowisko="formiernia", 
        Start=frw1_s2[0]-timedelta(minutes=65*2), End=frw1_s2[0],Ilosc="2szt"),    
    ]


In [8]:

tabela_operacje = pd.DataFrame(to1)

tabela_operacje["Czas operacji"] = tabela_operacje.End-tabela_operacje.Start
tabela_operacje.sort_values(by=["Operacja","Start"])
tabela_operacje

,Zamowienie,Produkt,Operacja,stanowisko,Start,End,Ilosc,Czas operacji
0,2023_1,Prod_1,zaladunek,zaladunek_1,2023-04-12 10:30:00.000000,2023-04-12 13:00:00,,0 days 02:30:00
1,2023_1,Prod_1,pakowanie,pakowanie h1s1,2023-04-12 04:30:00.000000,2023-04-12 10:30:00,1500kpl,0 days 06:00:00
2,2023_1,Prod_1,ciecie kartonu,ciecie kartonu,2023-04-11 16:57:41.538462,2023-04-12 04:30:00,1500kpl,0 days 11:32:18.461538
3,2023_1,Szklanka late 200ml,klejenie uszek reczne,klejnie uszek r3,2023-04-11 03:30:00.000000,2023-04-12 04:30:00,3000szt,1 days 01:00:00
4,2023_1,Szklanka late 200ml,klejenie uszek reczne,klejnie uszek r2,2023-04-11 03:30:00.000000,2023-04-12 04:30:00,3000szt,1 days 01:00:00
5,2023_1,Szklanka late 200ml,klejenie uszek reczne,klejnie uszek r1,2023-04-11 03:30:00.000000,2023-04-12 04:30:00,3000szt,1 days 01:00:00
6,2023_1,Pret 5mm bezbarwny,giecie preta,giecie preta,2023-04-10 05:00:00.000000,2023-04-11 03:30:00,9000szt,0 days 22:30:00
7,2023_1,Szklanka late 200ml,formowanie reczne,w1s2,2023-04-08 08:00:00.000000,2023-04-10 05:00:00,3000szt,1 days 21:00:00
8,2023_1,Szklanka late 200ml,formowanie reczne,w1s1,2023-04-08 08:00:00.000000,2023-04-10 05:00:00,3000szt,1 days 21:00:00
9,2023_1,Szklanka late 200ml,przygotowanie formy,formiernia,2023-04-08 05:50:00.000000,2023-04-08 08:00:00,2szt,0 days 02:10:00


In [9]:
tabela_operacje[tabela_operacje.Zamowienie == "2023_1"]["End"].max() - tabela_operacje[tabela_operacje.Zamowienie == "2023_1"]["Start"].min()

Timedelta('4 days 07:10:00')

In [10]:
fig = px.timeline(tabela_operacje, x_start="Start", x_end="End", y="stanowisko", color="Zamowienie",title="Operacje", hover_data=["Produkt", "Czas operacji", "Ilosc"])
# fig.update_yaxes(autorange="reversed")
# fig.add_vline(dt(2023,4,9,13), line_width=3, line_dash="dash", line_color="green")
fig.show()